### List of all products 

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import json
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
import requests

BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
ENDPOINT='/products'



url = BASE_URL + ENDPOINT

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}
productlist = []
listlength = 0
i = 0
for i in range(0, 10):
    query = {
        'page[number]': i,
        'page[size]': 200  # maximum
    }
    r = requests.get(url, params=query, headers=headers)
    if r.status_code == 200:
        print('Page ' + str(i) + ' loaded successfully.')

        for product in r.json()['data']:
            try:
                productlist.append(product['id'])
            except:
                pass

    if listlength == len(productlist):
        break
    #print('Product count: ' + str(len(productlist)))
    listlength = len(productlist)
    i += 1
    
'''
url = BASE_URL + ENDPOINT

query = {
  'page[number]': 1,
  'page[size]': 200
}

headers = {
  'Authorization': 'Token token=' + KEY,
  'Accept': 'application/json',
  'Content-Type': 'application/vnd.api+json',
  'Accept-Encoding': 'gzip,deflate'
}

r = requests.get(url, params=query, headers=headers)
print("Status: " + str(r.status_code))
if r.status_code == 200:
    print('Page loaded successfully.')
    print('Generated in ' + str(r.json().get('meta', {}).get('generated_in', -1)) + ' milliseconds.')
    print('Next page\'s URL is: ' + r.json()['links'].get('next', 'UNKNOWN'))
    print('Barcodes on this page:')
    for product in r.json()['data']:
        productlist = []
        for x in r.json()['data']:
            productlist.append(x['id'])
print(productlist)
'''

### Getting list of useful characteristics from id

In [ ]:
%%time
BASE_URL='https://www.openfood.ch/api/v3'
KEY='97bb347d26f3a8b39b4558827c3fcf56'
dInfo = {}
dIngredients = {}
dNutrients = {}

for PRODUCT_ID in productlist:
    ENDPOINT='/products/{}'.format(PRODUCT_ID)

    url = BASE_URL + ENDPOINT

    headers = {
      'Authorization': 'Token token=' + KEY,
      'Accept': 'application/json',
      'Content-Type': 'application/vnd.api+json',
      'Accept-Encoding': 'gzip,deflate'
    }

    r = requests.get(url, headers=headers)
    #print ('Status: ' + str(r.status_code))
    item = {}
    item_ing = {}
    item_nut = {}
    data = r.json()['data']
    if r.status_code == 200:
        try:
            item['name'] = item_ing['name'] = item_nut['name'] = data['display_name_translations'].popitem()
            item['id'] = item_ing['id'] = item_nut['id'] = data['id']
            #item['barcode'] = item_ing['barcode'] = item_nut['barcode'] = data['barcode']
            
            item['quantity'] = data['quantity']
            item['unit'] = data['unit']
            item['portion'] = data['portion_quantity']
            item['portion unit'] = data['portion_unit']
            item['alcohol'] = data['alcohol_by_volume']
            
            item_ing['ingredients'] = data['ingredients_translations'].popitem()
            
            nuts = {}
            for nut in data['nutrients'].keys():
                nuts[nut] = {'per_day':data['nutrients'][nut]['per_day'],
                             'per_hundred':data['nutrients'][nut]['per_hundred'],
                             'per_portion':data['nutrients'][nut]['per_portion']}
            item_nut['nutrients'] = nuts
            
            dInfo[data['barcode']] = item    
            dIngredients[data['barcode']] = item_ing
            dNutrients[data['barcode']] = item_nut
        except:
            pass

### Translate ingredients

In [ ]:
%%time
from watson_developer_cloud import LanguageTranslatorV2 as LanguageTranslator

language_translator = LanguageTranslator(username='a69f77dc-9248-4069-85b5-1ac1ef6bfd7d',password='UQznP8xcKtuq')

count = 0
print("%s/%s items done..."%(count, len(dIngredients)))
remove = []
for barcode, tmp in dIngredients.items():
    try:        
        ingredients_ = ' '.join(tmp['ingredients'][1].splitlines())
        language_json = language_translator.identify(ingredients_)
        language = pd.DataFrame(language_json['languages']).sort_values(by=['confidence'], ascending=False)
        source_lang = language.iloc[0, 1]
        if source_lang != "en":
            translation = language_translator.translate(text=ingredients_, source=source_lang, target='en')
            dIngredients[barcode]['ingredients'] = "en_translated, " + json.dumps(translation, indent=2, ensure_ascii=False)
        else:
            dIngredients[barcode]['ingredients'] = "en_kept, " + ingredients_ 
                        
    except:
        #print(source_lang, ingredients_)
        remove.append(barcode)
        
    count += 1
    if count % 50 == 0:
        print("%s/%s items done!"%(count, len(dIngredients)))

# remove the ones (from everywhere) which couldn't be translated
for barcode in remove:
    del dInfo[barcode]
    del dIngredients[barcode]
    del dNutrients[barcode]

In [ ]:
# dump dictionaries to json files (load it in with `json.load(fName)`)

with open("dInfo.json", "w") as f_:
    json.dump(dInfo, f_)
with open("dIngredients.json", "w") as f_:
    json.dump(dIngredients, f_)
with open("dNutrients.json", "w") as f_:
    json.dump(dNutrients, f_)

### Calculate climate impact!

In [9]:
import json

with open("dInfo.json", "r") as f_:
    dInfo = json.load(f_)
with open("dIngredients.json", "r") as f_:
    dIngredients = json.load(f_)
with open("dNutrients.json", "r") as f_:
    dNutrients = json.load(f_)

In [10]:
from food_db import build_database
ingredients_to_impact = build_database()

In [11]:
import re
ingredient_dict = {}

for item in list(dIngredients.items()):
    lItems = item[1]['ingredients'].split(',')[1:]
    quantity_dict = {}
    estimate = 50
    for l in lItems:
        percentage = re.findall(r'\d+%', l)
        if(len(percentage) == 0):
            quantity_dict[l] = estimate
        else:
            quantity_dict[l] = percentage[0].rstrip('%')  # TODO: handle second instance...
        estimate = estimate/2
    ingredient_dict[item[0]] = quantity_dict

In [12]:
final_dict = {}
for bar_code in list(ingredient_dict):
    temp_dict = {}
    temp_dict['ingredients'] = ingredient_dict[bar_code]
    temp_dict['quantity'] = dInfo[bar_code]['quantity']
    temp_dict['unit'] = dInfo[bar_code]['unit']
    final_dict[bar_code] = temp_dict
    

In [13]:
%%time
import difflib

footprint_dict = {}
for bar_code, item in final_dict.items():
    footprint = 0
    ingredients = item['ingredients']
    quantity = float(item['quantity'])
    for ing_key, ing_quantity in ingredients.items():
        #print(ing_key)
        scores = []
        keys = []
        for key, value in ingredients_to_impact.items():
            keys.append(key)
            scores.append(difflib.SequenceMatcher(None, ing_key, key).ratio())
            
        matched_score = ingredients_to_impact[keys[scores.index(max(scores))]]
        #print(quantity, ing_quantity, matched_score)
        footprint += float(quantity) * float(ing_quantity)*0.01 * float(matched_score)/1000.
    footprint_dict[bar_code] = footprint

CPU times: user 9.86 s, sys: 0 ns, total: 9.86 s
Wall time: 9.87 s


In [7]:
with open("barcode_to_impact.json", "w") as f_:
    json.dump(footprint_dict, f_)

In [8]:
with open("barcode_to_impact.csv", "w") as f_:
    for barcode, footprint in footprint_dict.items():
        f_.write("%s;+%.3f\n"%(barcode, footprint))